# Biomethane-to-Ammonia - Sensitivity analysis

This notebook performs sensitivity analysis using the presamples package and export the results.

**IMPORTANT**:

In order to avoid the decoding presample json file fails, you need to install the latest presample version, i.e. 0.2.8.

However, the package has not been updated on pypi. To have version 0.2.8, you must install from source:

*python -m pip install git+https://github.com/PascalLesage/presamples.git@master*

Further information here: https://github.com/PascalLesage/presamples/issues/72

In [1]:
%run _imports.ipynb

## Methane leakage sensitivity analysis

In [21]:
assessed_ds = [a for a in bw.Database(LCI_DB) if "ammonia production, liquid, bio-ammonia from biomethane" in a['name'] and a['location'] == 'RER']
assessed_ds

['ammonia production, liquid, bio-ammonia from biomethane w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane, syngas w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane, syngas w/ CCS, heating w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS, heating w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane' (kilogram, RER, None)]

In [22]:
scenario_file = Path(DATA_DIR / "SA_methane leakage_for presample.xlsx")
SA_methane_leakage_results = {}

for ds in assessed_ds:
    scenario = [key for key, value in INVENTORIES.items() if value == (ds["name"], ds["reference product"])][0]

    # Read the excel file to get the scenario data
    scenario_label, scenario_data = results_analysis.read_ps_scenario_data(scenario_file, dbs=[ECOINVENT_DB, LCI_DB])

    # Create PS package with indices + samples matrix
    ps_packagename = "ps_v1"
    ps_filepath_v1 =  results_analysis.make_ps_package(scenario_data, scenario_label, ps_packagename)

    # Calculate impacts with presample
    SA_lca_results = results_analysis.calculate_impacts_with_ps(ps_filepath_v1, scenario_label, ds, IPCC_METHOD)

    SA_methane_leakage_results.update(
        {scenario: {
            key: value['Climate change, GWP 100a'] for key, value in SA_lca_results.to_dict().items()
            }
        }
    )


 ps_id, filepath: 9f672eeb48b648da8081b7296a65b175 C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\9f672eeb48b648da8081b7296a65b175


c:\Users\istrateir\Anaconda3\envs\iri\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)



 ps_id, filepath: e9b0616123074917a2fef11c3cd21cc5 C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\e9b0616123074917a2fef11c3cd21cc5


c:\Users\istrateir\Anaconda3\envs\iri\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)



 ps_id, filepath: 687632548a24436ab029d0977ad83226 C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\687632548a24436ab029d0977ad83226


...

c:\Users\istrateir\Anaconda3\envs\iri\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)



 ps_id, filepath: a8e7e90af0f84f748430203921c206e8 C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\a8e7e90af0f84f748430203921c206e8


c:\Users\istrateir\Anaconda3\envs\iri\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)



 ps_id, filepath: 3118b6e72cef42b48fac3f4f4b7c270c C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\3118b6e72cef42b48fac3f4f4b7c270c


c:\Users\istrateir\Anaconda3\envs\iri\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [23]:
SA_methane_leakage_results_df = pd.DataFrame(SA_methane_leakage_results).T

# Move default column to he first position
default_col = SA_methane_leakage_results_df.pop("default")
SA_methane_leakage_results_df.insert(0, default_col.name, default_col)

In [ ]:
# Save results to csv
#SA_methane_leakage_results_df.to_csv(DATA_DIR / "results" / f"Fig S1 Sensitivity analysis carbon footprint ammonia methane leakage_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Scenario")

## Biogas upgrading technology sensitivity analysis

In [25]:
ds_for_sa = [a for a in bw.Database(LCI_DB) if "ammonia production, liquid, bio-ammonia from biomethane" in a['name'] and a['location'] == 'RER']
ds_for_sa

['ammonia production, liquid, bio-ammonia from biomethane' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane, syngas w/ CCS, heating w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane, syngas w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane w/ CCS' (kilogram, RER, None),
 'ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS, heating w/ CCS' (kilogram, RER, None)]

In [26]:
scenario_file = Path(DATA_DIR / "SA_upgrading technology_presamples.xlsx")

SA_upgradingtechnology_results = {}

for ds in ds_for_sa:
    scenario = [key for key, value in INVENTORIES.items() if value == (ds["name"], ds["reference product"])][0]

    # Read the excel file to get the scenario data
    scenario_label, scenario_data = results_analysis.read_ps_scenario_data(scenario_file, dbs=[ECOINVENT_DB, LCI_DB])

    # Create PS package with indices + samples matrix
    ps_packagename = "ps_v1"
    ps_filepath_v1 =  results_analysis.make_ps_package(scenario_data, scenario_label, ps_packagename)

    # Calculate impacts with presample
    SA_lca_results = results_analysis.calculate_impacts_with_ps(ps_filepath_v1, scenario_label, ds, LCIA_METHODS)

    SA_upgradingtechnology_results.update(
        {
            scenario: SA_lca_results.to_dict()
        }
    )


 ps_id, filepath: ffe45d3b774d40bcae26b3b8adc9006b C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\ffe45d3b774d40bcae26b3b8adc9006b

 ps_id, filepath: a69c8c1f2aa54e1c87757ea6940e5f91 C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\a69c8c1f2aa54e1c87757ea6940e5f91

 ps_id, filepath: b81f05f6d1944ba282950fa893cde1df C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\b81f05f6d1944ba282950fa893cde1df

 ps_id, filepath: 95602da57eae43de8b43c8a6d194e8a6 C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\95602da57eae43de8b43c8a6d194e8a6

 ps_id, filepath: 58f89ab0671145a08681c7f56f33f9c6 C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\58f89ab0671145a08681c7f56f33f9c6

 ps_id, filepath: 6a06d6278f6b449b9813d84ffcb530d0 C:\Users

In [27]:
# Results for carbon footprint
SA_upgradingtechnology_carbon_footprint = {}
SA_upgradingtechnology_carbon_footprint.update(
    {
        scenario: {key: value['Climate change, GWP 100a'] for key, value in SA_upgradingtechnology_results[scenario].items()
                }
                for scenario in SA_upgradingtechnology_results.keys()
        }
)

In [ ]:
# Save results to csv
#pd.DataFrame(SA_upgradingtechnology_carbon_footprint).T.to_csv(DATA_DIR / "results" / f"Fig S3 Sensitivity analysis carbon footprint ammonia upgrading technology_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Scenario")

## One-at-a-time (OAT) sensitivity analysis

In [2]:
assessed_ds = [a for a in bw.Database(LCI_DB) if a['name'] == "ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS, heating w/ CCS"
                                and a['location'] == 'RER'][0]

assessed_ds

'ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS, heating w/ CCS' (kilogram, RER, None)

In [3]:
product_for_oat = [
    "biogas",
    "animal manure",
    "oat forage, sequential crop",
    "biomethane, 24 bar",
    "heat, district or industrial",
    "carbon dioxide capture, chemical absorption, with transport and storage, 200 km pipeline sotage 1000m",
    "carbon dioxide transport and storage, 200 km pipeline, storage 1000m",
  #  "pipeline for carbon dioxide transport, supercritical, 200km w/o recompression",
  #  "drilling, deep borehole/m"
]

ds_for_oat = [a for a in bw.Database(LCI_DB) if a["location"] == "RER" and
                                                a["reference product"] in product_for_oat and
                                                "market for" not in a["name"]]

# Add assessed dataset
ds_for_oat += [assessed_ds]

# Drop not relevant datasets
for item in ds_for_oat:
    if item["reference product"] in ["biomethane, 24 bar", "heat, district or industrial"]:
        if ("w/ CCS") not in item["name"]:
          ds_to_drop = [a for a in ds_for_oat if a["name"] == item["name"]][0]
          ds_for_oat.remove(ds_to_drop)

In [4]:
perturbation_analysis_results = results_analysis.perturbation_analysis_with_ps(assessed_ds, ds_for_oat, [ECOINVENT_DB, LCI_DB], IPCC_METHOD)


 ps_id, filepath: b5dad24712f34752871e06e586fb979b C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\b5dad24712f34752871e06e586fb979b

 ps_id, filepath: 8d22322766e4445c8daeb96dff2a40cb C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_work.4ee317080836a482ef8d98dd52304958\presamples\8d22322766e4445c8daeb96dff2a40cb


In [94]:
# Save results to csv
#perturbation_analysis_results.to_csv(DATA_DIR / "results" / f"Fig S6 Perturbation analysis carbon footprint scenario full CCS_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Scenario")